In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

### **Ejercicios de Análisis de Datos Biosanitarios**

### 1. Análisis de Datos Tabulares

### Ejercicio 1.1: Perfil Epidemiológico de Enfermedades Cardiovasculares

Descripción: Analizar un conjunto de datos de pacientes para identificar 
factores de riesgo y patrones relacionados con enfermedades cardiovasculares. 

### Instrucciones: 

### 1. Cargar un conjunto de datos de historias clínicas con las siguientes variables:
- Edad 
- Género 
- Nivel de colesterol 
- Presión arterial 
- Índice de masa corporal (IMC) 
- Historial de diabetes 
- Incidencia de enfermedad cardiovascular

In [2]:
'''# Generacion de datos sinteticos
def generar_datos_cardiovasculares(n_muestras=1000):
    np.random.seed(42)
    
    # Generación de variables
    edad = np.random.normal(55, 15, n_muestras)
    genero = np.random.choice(['Masculino', 'Femenino'], n_muestras)
    colesterol = np.random.normal(200, 40, n_muestras)
    presión_arterial = np.random.normal(130, 20, n_muestras)
    imc = np.random.normal(27, 5, n_muestras)
    diabetes = np.random.choice([0,1], n_muestras, p=[0.7, 0.3])
    
    # Cálculo de riesgo cardiovascular
    riesgo = (
        (edad > 50) * 0.3 +
        (genero == 'Masculino') * 0.2 +
        (colesterol > 240) * 0.25 +
        (presión_arterial > 140) * 0.2 +
        (imc > 30) * 0.15 +
        (diabetes == 1) * 0.3
        )
    
    # COnvertir el DataFrame
    df = pd.DataFrame ({
        'Edad': edad,
        'Género': genero,
        'Colesterol': colesterol,
        'Presión arterial': presión_arterial,
        'IMC': imc,
        'Diabetes': diabetes,
        'Riesgo Cardiovascular': riesgo
        })

    return df

datos_cv = generar_datos_cardiovasculares()'''

"# Generacion de datos sinteticos\ndef generar_datos_cardiovasculares(n_muestras=1000):\n    np.random.seed(42)\n    \n    # Generación de variables\n    edad = np.random.normal(55, 15, n_muestras)\n    genero = np.random.choice(['Masculino', 'Femenino'], n_muestras)\n    colesterol = np.random.normal(200, 40, n_muestras)\n    presión_arterial = np.random.normal(130, 20, n_muestras)\n    imc = np.random.normal(27, 5, n_muestras)\n    diabetes = np.random.choice([0,1], n_muestras, p=[0.7, 0.3])\n    \n    # Cálculo de riesgo cardiovascular\n    riesgo = (\n        (edad > 50) * 0.3 +\n        (genero == 'Masculino') * 0.2 +\n        (colesterol > 240) * 0.25 +\n        (presión_arterial > 140) * 0.2 +\n        (imc > 30) * 0.15 +\n        (diabetes == 1) * 0.3\n        )\n    \n    # COnvertir el DataFrame\n    df = pd.DataFrame ({\n        'Edad': edad,\n        'Género': genero,\n        'Colesterol': colesterol,\n        'Presión arterial': presión_arterial,\n        'IMC': imc,\n   

### 2. Realizar las siguientes tareas de análisis: 
- Calcular estadísticas descriptivas 
- Identificar correlaciones entre variables 
- Segmentar la población por grupos de riesgo 
- Crear visualizaciones para comparar factores 

In [3]:
'''def analisis_perfil_cardiovascular(df):
    # Estadísticas descriptivas
    print('Estadísticas descriptivas')
    print(df.describe())
    
    # Correlaciónes
    correlaciones = df[['Edad', 'Colesterol', 'Presión arterial', 'IMC']].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlaciones, annot=True, cmap='coolwarm')
    plt.title('Matriz de correlaciones')
    plt.show()
    

    # Segmentar los grupos de riesgo
    features = ['Edad', 'Colesterol', 'Presión arterial', 'IMC']
    scaler = StandardScaler()
    datos_escalados = scaler.fit_transform(df[features])
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['Grupo de Riesgo'] = kmeans.fit_predict(datos_escalados)


    # Visualización de grupos de riesgo
    plt.figure(figsize=(12, 6))
    sns.scatterplot(data=df, x='Edad', y='Colesterol', hue='Grupo de Riesgo', palette='viridis')
    plt.title('Grupo de Riesgo Cardiovascular')
    plt.show()
                
    # Analisis de riesgo de género
    riesgo_por_genero = df.groupby('Género')['Riesgo Cardiovascular'].mean()
    print('\nRiesgo Cardiovascular por Género')
    print(riesgo_por_genero)
    
    
    # Prueba Estadística de diferencias de riesgo por género
    riesgo_masculino = df[df['Género'] == 'Masculino']['Riesgo Cardiovascular']
    riesgo_femenino = df[df['Género'] == 'Femenino']['Riesgo Cardiovascular']
    
    t_statistic, p_value = stats.ttest_ind(riesgo_masculino, riesgo_femenino)
    print(f'\nPrueba t de diferencia de riesgo por género')
    print(f'Estadístico t: {t_statistic}')
    print(f'Valor p: {p_value}')
    
    
                                        
# Ejecutar analisis 
analisis_perfil_cardiovascular(datos_cv)'''

"def analisis_perfil_cardiovascular(df):\n    # Estadísticas descriptivas\n    print('Estadísticas descriptivas')\n    print(df.describe())\n    \n    # Correlaciónes\n    correlaciones = df[['Edad', 'Colesterol', 'Presión arterial', 'IMC']].corr()\n    plt.figure(figsize=(10, 8))\n    sns.heatmap(correlaciones, annot=True, cmap='coolwarm')\n    plt.title('Matriz de correlaciones')\n    plt.show()\n    \n\n    # Segmentar los grupos de riesgo\n    features = ['Edad', 'Colesterol', 'Presión arterial', 'IMC']\n    scaler = StandardScaler()\n    datos_escalados = scaler.fit_transform(df[features])\n    kmeans = KMeans(n_clusters=3, random_state=42)\n    df['Grupo de Riesgo'] = kmeans.fit_predict(datos_escalados)\n\n\n    # Visualización de grupos de riesgo\n    plt.figure(figsize=(12, 6))\n    sns.scatterplot(data=df, x='Edad', y='Colesterol', hue='Grupo de Riesgo', palette='viridis')\n    plt.title('Grupo de Riesgo Cardiovascular')\n    plt.show()\n                \n    # Analisis de rie

### Ejercicio 1.2: Análisis de Ensayo Clínico de Nuevo Medicamento 

Descripción: Evaluar la eficacia de un nuevo tratamiento comparando grupos de 
intervención y control.

Instrucciones: 

1. Importar datos de un ensayo clínico con columnas: 
- ID de paciente 
- Grupo (intervención/control) 
- Dosis del medicamento 
- Marcadores biológicos 
- Efectos secundarios 
- Tiempo de recuperación 

In [12]:
'''# Generación de datos sintéticos
def generar_datos_ensayo_clinico(n_muestras=500):
    np.random.seed(42)
    
    # Variables
    grupo = np.random.choice(['Intervencion', 'Control'], n_muestras)
    dosis = np.where(grupo == 'Intervencion',
                     np.randomuniform(50, 200, n_muestras),
                     np.zeros(n_muestras))
    
    # Marcadores biologicos simulados
    marcador_base = np.random.normal(100, 20, n_muestras)
    marcador_final = np.where(
        grupo == 'Intervencion',
        marcador_base * (1 - np.random.uniform(0.1, 0.3, n_muestras)),
        marcador_base
    )
    
    # Efectos secundarios
    efectos_secundarios = np.random.binomial(1, 0.2 if grupo == 'Intervencion' else 0.05, n_muestras)
    tiempo_recuperacion = np.random.normal(
        14 if grupo == 'Intervencion' else 21,
        3,
        n_muestras)
    
    df = pd.DataFrame({
        'ID': range(1, n_muestras + 1),
        'Grupo': grupo,
        'Dosis': dosis,
        'Marcador_inicial' : marcador_base,
        'Marcador_final': marcador_final,
        'Efectos_secundarios': efectos_secundarios,
        'Tiempo_recuperacion': tiempo_recuperacion
    })
    
    return df

# Generar los datos

datos_ensayo= generar_datos_ensayo_clinico()

# Analisis ensayo
def analisis_ensayo_clinico(df):
    # Analisis comparativo de grupos
    comparacion_grupos = df.groupby('Grupo').agg({
        'Marcador_inicial': 'mean',
        'Marcador_final': 'mean',
        'Efectos_secundarios': 'mean',
        'Tiempo_recuperacion': 'mean'
    })
    print('Comparacionde grupos')
    print(comparacion_grupos)

# Ejecutar analisis
analisis_ensayo_clinico(datos_ensayo)'''

# Generación de datos sintéticos
def generar_datos_ensayo_clinico(n_muestras=500):
    np.random.seed(42)
    # Variables
    grupo = np.random.choice(['Intervencion', 'Control'], n_muestras)
    dosis = np.where(grupo == 'Intervencion',
                     np.random.uniform(50, 200, n_muestras),
                     np.zeros(n_muestras))
    # Marcadores biológicos simulados
    marcador_base = np.random.normal(100, 20, n_muestras)
    marcador_final = np.where(
        grupo == 'Intervencion',
        marcador_base * (1 - np.random.uniform(0.1, 0.3, n_muestras)),
        marcador_base
    )
    # Efectos secundarios
    efectos_secundarios = np.where(grupo == 'Intervencion', 0.2, 0.05)
    efectos_secundarios = np.random.binomial(1, efectos_secundarios)
    tiempo_recuperacion = np.where(
        grupo == 'Intervencion',
        np.random.normal(14, 3, n_muestras),
        np.random.normal(21, 3, n_muestras)
    )
    
    df = pd.DataFrame({
        'ID': range(1, n_muestras + 1),
        'Grupo': grupo,
        'Dosis': dosis,
        'Marcador_inicial': marcador_base,
        'Marcador_final': marcador_final,
        'Efectos_secundarios': efectos_secundarios,
        'Tiempo_recuperacion': tiempo_recuperacion
    })
    return df
# Generar los datos
datos_ensayo = generar_datos_ensayo_clinico()
# Análisis ensayo
def analisis_ensayo_clinico(df):
    # Análisis comparativo de grupos
    comparacion_grupos = df.groupby('Grupo').agg({
        'Marcador_inicial': 'mean',
        'Marcador_final': 'mean',
        'Efectos_secundarios': 'mean',
        'Tiempo_recuperacion': 'mean'
    })
    print('Comparación de grupos')
    print(comparacion_grupos)
# Ejecutar análisis
analisis_ensayo_clinico(datos_ensayo)

Comparación de grupos
              Marcador_inicial  Marcador_final  Efectos_secundarios  \
Grupo                                                                 
Control             100.975091      100.975091             0.039062   
Intervencion        102.170947       81.382710             0.213115   

              Tiempo_recuperacion  
Grupo                              
Control                 20.986556  
Intervencion            14.203042  


2. Tareas a realizar: 
- Realizar pruebas estadísticas de significancia 
- Calcular tasas de respuesta al tratamiento 
- Analizar distribución de efectos secundarios 
- Crear intervalos de confianza para los resultados 

### Ejercicio 1.3: Estudio Nutricional Longitudinal

**Descripción**: Analizar cambios en parámetros nutricionales de un grupo de 
pacientes.

**Instrucciones**:

1. Cargar datos con información: 
- Nombre del participante 
- Edad 
- Peso 
- Índice de masa corporal 
- Consumo calórico 
- Niveles de micronutrientes 
- Actividad física

2. Desarrollar análisis: 
- Identificar tendencias de cambio 
- Calcular promedios y desviaciones estándar 
- Realizar agrupaciones por características demográficas 
- Generar informes de progresión individual 

### Ejercicio 1.4: Base de Datos Genómica 

**Descripción**: Analizar variantes genéticas asociadas a una condición médica 
específica. 

**Instrucciones**: 

1. Manejar conjunto de datos genómicos con: 
- ID de muestra 
- Marcadores genéticos 
- Presencia/ausencia de mutación 
- Información clínica asociada 
- Resultado de condición médica 

2. Tareas de análisis: 
- Identificar frecuencia de variantes 
- Realizar análisis de asociación genética 
- Crear visualizaciones de distribución genética 
- Calcular riesgos relativos 

### Ejercicio 1.5: Registro de Eventos Hospitalarios

**Descripción**: Analizar eficiencia y patrones en un registro de eventos 
hospitalarios.

**Instrucciones**:

1. Trabajar con datos de: 
- Número de ingreso 
- Fecha de admisión/alta 
- Especialidad médica 
- Duración de estancia 
- Diagnóstico principal 
- Procedimientos realizados

2. Realizar análisis: 
- Calcular tiempos de estancia promedio 
- Identificar especialidades con mayor carga 
- Analizar distribución de diagnósticos 
- Crear indicadores de eficiencia hospitalaria 


### 2. Análisis de Grafos

### Ejercicio 2.1: Red de Transmisión Epidemiológica

Descripción: Modelar y analizar la propagación de una enfermedad infecciosa. 

Instrucciones: 

1. Construir grafo de transmisión con: 
- Nodos: Individuos 
- Aristas: Contactos/Transmisión 
- Atributos: Edad, estado de salud, localización 

2. Tareas de análisis: 
- Calcular centralidad de nodos 
- Identificar "superspreaders" 
- Simular patrones de propagación 
- Visualizar estructura de red 

### Ejercicio 2.2: Interacciones Proteicas 

Descripción: Analizar red de interacciones proteicas en una vía metabólica. 

Instrucciones: 

1. Crear grafo de interacciones con: 
- Nodos: Proteínas 
- Aristas: Interacciones/Modificaciones 
- Atributos: Función biológica 

2. Desarrollar análisis: 
- Identificar proteínas clave 
- Calcular grados de conectividad 
- Analizar comunidades de proteínas 
- Visualizar estructura de red

### Ejercicio 2.3: Red de Colaboración Científica 

Descripción: Estudiar colaboraciones en investigación biomédica. 

Instrucciones:

1. Construir grafo de: 
- Nodos: Investigadores 
- Aristas: Publicaciones conjuntas 
- Atributos: Especialidad, institución 

2. Realizar análisis: 
- Calcular índices de centralidad 
- Identificar investigadores clave 
- Analizar comunidades de investigación 
- Visualizar estructura de colaboración 

### Ejercicio 2.4: Red de Referencia Hospitalaria 

Descripción: Modelar sistema de referencias entre centros de salud. 

Instrucciones:

1. Crear grafo de: 
- Nodos: Centros de salud 
- Aristas: Referencias/Derivaciones 
- Atributos: Especialidades, complejidad 

2. Tareas de análisis: 
- Identificar centros centrales 
- Calcular flujos de referencia 
- Analizar eficiencia del sistema 
- Visualizar estructura de red 


### Ejercicio 2.5: Red de Transmisión Genética

Descripción: Analizar árbol genealógico para estudios de herencia. 

Instrucciones:

1. Construir grafo de: 
- Nodos: Individuos 
- Aristas: Relaciones familiares 
- Atributos: Características genéticas

2. Desarrollar análisis: 
- Identificar patrones de herencia 
- Calcular probabilidades genéticas 
- Visualizar árbol genealógico 
- Analizar riesgos de transmisión

### 3. Análisis de Datos Textuales

Ejercicio 3.1: Análisis de Historias Clínicas

Descripción: Extraer información de registros médicos textuales.

Instrucciones: 

1. Recopilar conjunto de: 
- Informes médicos 
- Notas de evolución 
- Resúmenes de alta 

2. Realizar procesamiento: 
- Limpieza de texto 
- Extracción de entidades médicas 
- Análisis de sentimiento 
- Identificación de diagnósticos

### Ejercicio 3.2: Minería de Literatura Científica

Descripción: Analizar publicaciones biomédicas.

Instrucciones: 

1. Recopilar corpus de: 
- Artículos científicos 
- Resúmenes de investigación 
- Revisiones sistemáticas 

2. Desarrollar análisis: 
- Extracción de palabras clave 
- Identificación de temas 
- Análisis de co-ocurrencia 
- Visualización de tendencias

### Ejercicio 3.3: Análisis de Consentimientos Informados

Descripción: Procesar documentos de consentimiento. 

Instrucciones: 

1. Trabajar con: 
- Modelos de consentimiento 
- Variantes de documentos 
- Versiones en diferentes idiomas 

2. Realizar tareas: 
- Normalización de texto 
- Extracción de cláusulas 
- Análisis comparativo 
- Identificación de variaciones

### Ejercicio 3.4: Revisión de Reportes de Eventos Adversos

Descripción: Analizar reportes de farmacovigilancia.

Instrucciones:

1. Recopilar: 
- Reportes de efectos secundarios 
- Descripciones de incidentes 
- Información de pacientes 

2. Desarrollar análisis: 
- Clasificación de eventos 
- Extracción de síntomas 
- Identificación de patrones 
- Generación de alertas 

### Ejercicio 3.5: Análisis de Comunicaciones de Pacientes

Descripción: Procesar comunicaciones en plataformas de salud.

Instrucciones:

1. Trabajar con: 
- Mensajes de pacientes 
- Consultas en línea 
- Foros de salud

2. Realizar tareas: 
- Clasificación de consultas 
- Análisis de sentimiento 
- Detección de necesidades 
- Generación de insights 

### 4. Análisis de Series Temporales

Ejercicio 4.1: Monitoreo de Indicadores Epidemiológicos 

Descripción: Analizar evolución de enfermedades. 

Instrucciones: 

1. Recopilar datos de: 
- Casos confirmados 
- Hospitalizaciones 
- Recuperaciones 
- Fallecimientos 

2. Realizar análisis: 
- Descomposición de series 
- Predicción de tendencias 
- Detección de patrones estacionales 
- Modelado de escenarios 


### Ejercicio 4.2: Seguimiento de Parámetros Fisiológicos 

Descripción: Monitorear variables de pacientes.

Instrucciones:

1. Trabajar con: 
- Registros de presión arterial 
- Niveles de glucosa 
- Frecuencia cardíaca 
- Peso corporal 

2. Desarrollar análisis: 
- Identificación de tendencias 
- Detección de anomalías 
- Predicción de cambios 
- Generación de alertas 

### Ejercicio 4.3: Evolución de Tratamientos Farmacológicos

Descripción: Seguimiento de respuesta a medicamentos.

Instrucciones: 

1. Recopilar datos de: 
- Dosis administradas 
- Marcadores biológicos 
- Respuesta al tratamiento 
- Efectos secundarios 

2. Realizar tareas: 
- Análisis de progresión 
- Modelado de respuesta 
- Identificación de patrones 
- Optimización de tratamientos 

### Ejercicio 4.4: Estudio de Variabilidad Genética 

Descripción: Analizar cambios genéticos en el tiempo. 

Instrucciones: 


1. Trabajar con: 
- Secuencias genéticas 
- Mutaciones 
- Expresión génica 
- Marcadores evolutivos 

2. Desarrollar análisis: 
- Identificación de cambios 
- Modelado de evolución 
- Predicción de variantes 
- Estudio de trayectorias 

### Ejercicio 4.5: Registro de Parámetros Ambientales en Salud

Descripción: Monitorear impacto ambiental.

Instrucciones: 

1. Recopilar datos de: 
- Contaminación atmosférica 
- Temperatura 
- Humedad 
- Índices de enfermedades

2. Realizar tareas: 
- Correlación ambiente-salud 
- Predicción de impactos 
- Identificación de patrones 
- Modelado de escenarios

### 5. Análisis de Imágenes

Ejercicio 5.1: Diagnóstico por Imágenes Médicas

Descripción: Procesamiento de radiografías.

Instrucciones: 

1. Trabajar con: 
- Radiografías de tórax 
- Imágenes de resonancia 
- Tomografías 
- Marcadores de lesiones 

2. Realizar tareas: 
- Segmentación de imágenes 
- Detección de anomalías -
- Clasificación de lesiones 
- Cuantificación de hallazgos 

### Ejercicio 5.2: Análisis Histológico 

Descripción: Procesamiento de imágenes microscópicas. 

Instrucciones: 

1. Recopilar: 
- Preparaciones histológicas 
- Cortes celulares 
- Imágenes de tejidos 
- Marcadores tumorales

2. Desarrollar análisis: 
- Segmentación celular 
- Clasificación de estructuras 
- Cuantificación de células 
- Detección de patologías 

### Ejercicio 5.3: Seguimiento de Cicatrización

Descripción: Monitoreo de heridas.

Instrucciones: 

1. Trabajar con: 
- Fotografías de heridas 
- Imágenes secuenciales 
- Marcadores de evolución 
- Registros de curación

2. Realizar tareas: 
- Medición de área 
- Análisis de color 
- Detección de cambios 
- Predicción de cicatrización 

### Ejercicio 5.4: Análisis de Biomarcadores Visuales

Descripción: Procesamiento de imágenes de biomarcadores.

Instrucciones:

1. Recopilar: 
- Imágenes de marcadores 
- Tinciones específicas 
- Biomarcadores fluorescentes 
- Registros de expresión 

2. Desarrollar análisis: 
- Cuantificación de expresión 
- Segmentación de marcadores 
- Clasificación de patrones 
- Correlación con diagnósticos 

### Ejercicio 5.5: Análisis Genético por Imagen 

Descripción: Procesamiento de cariotipo.

Instrucciones:

1. Trabajar con: 
- Imágenes de cromosomas 
- Preparaciones de cariotipo 
- Marcadores cromosómicos 
- Registros genéticos 

2. Realizar tareas: 
- Identificación cromosómica 
- Detección de anomalías 
- Cuantificación de variantes 
- Análisis de estructuras 


### Próximos Pasos 

**Para cada ejercicio, se recomienda**:
   - Utilizar herramientas como Python (pandas, numpy, scikit-learn) 
   - Emplear librerías especializadas (scipy, matplotlib) 
   - Aplicar técnicas de machine learning 
   - Validar resultados con expertos del área 